In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def bayesian_ridge_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    n_iter=300,
    alpha_1=1e-6,
    alpha_2=1e-6,
    lambda_1=1e-6,
    lambda_2=1e-6,
    compute_scores=True,
    random_state=None
):
    """
    Bayesian Ridge Regression for time series forecasting with:
    - Probabilistic uncertainty estimates
    - Automatic complexity control
    - Lag features only
    """
    # Create directory structure
    os.makedirs('BayesianRidgeRegression', exist_ok=True)
    
    # 1. Stationarity and Differencing
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:
            d = 1
            series = series.diff().dropna()

    # 2. Feature Engineering - Lag features
    df = pd.DataFrame({'y': series.astype(float)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(float)
    df = df.dropna()

    # 3. Train-Test Split
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 4. Model Training with Bayesian Ridge
    model = BayesianRidge(
        n_iter=n_iter,
        alpha_1=alpha_1,
        alpha_2=alpha_2,
        lambda_1=lambda_1,
        lambda_2=lambda_2,
        compute_score=compute_scores,
    )
    model.fit(X_train, y_train)

    # 5. Forecasting with Uncertainty
    pred_test, pred_test_std = model.predict(X_test, return_std=True)

    # 6. Inverse Differencing
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 7. Create forecast DataFrame with uncertainty
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast_mean': pred_test,
        'forecast_std': pred_test_std
    })

    # 8. Save forecasts
    forecast_csv_path = os.path.join('BayesianRidgeRegression', 'bayesian_forecasts.csv')
    write_header = not os.path.exists(forecast_csv_path)
    forecast_df.to_csv(forecast_csv_path, mode='a', header=write_header, index=False)

    # 9. Calculate metrics and uncertainty statistics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast_mean']))
    log_marginal_likelihood = model.scores_[-1] if compute_scores else np.nan
    
    # Calculate coefficient statistics
    coef_means = model.coef_
    coef_stds = np.sqrt(np.diag(model.sigma_))
    
    metrics_df = pd.DataFrame([{
        'district': district_name,
        'rmse': rmse,
        'log_marginal_likelihood': log_marginal_likelihood,
        'coefficient_means': coef_means.tolist(),
        'coefficient_stds': coef_stds.tolist(),
        'intercept_mean': model.intercept_,
        'intercept_std': np.nan,
        'effective_params': model.lambda_ / model.alpha_,
        'differencing': d,
        'n_iter': n_iter
    }])

    metrics_csv_path = os.path.join('BayesianRidgeRegression', 'bayesian_metrics.csv')
    write_header_metrics = not os.path.exists(metrics_csv_path)
    metrics_df.to_csv(metrics_csv_path, mode='a', header=write_header_metrics, index=False)

    # 10. Plotting with uncertainty bands
    plt.figure(figsize=(12, 6))
    plt.plot(original_series.index, original_series.values, label='Original Series')
    plt.plot(forecast_df['date'], forecast_df['forecast_mean'], label='Mean Forecast', color='darkorange')
    plt.fill_between(
        forecast_df['date'],
        forecast_df['forecast_mean'] - 1.96*forecast_df['forecast_std'],
        forecast_df['forecast_mean'] + 1.96*forecast_df['forecast_std'],
        color='orange',
        alpha=0.2,
        label='95% Confidence'
    )
    plt.title(
        f'Bayesian Ridge Forecast for {district_name}\n'
        f'RMSE: {rmse:.2f} | Log ML: {log_marginal_likelihood:.2f}'
    )
    plt.legend()
    plt.savefig(os.path.join('BayesianRidgeRegression', f'bayesian_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'model': model,
        'coef_means': coef_means,
        'coef_stds': coef_stds,
        'intercept_mean': model.intercept_,
        'intercept_std': np.nan
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I55"].asfreq('MS')
        
        results = bayesian_ridge_regression_lags_only(
            ts,
            district,
            max_lags=4,
            n_iter=500,
            alpha_1=1e-5,
            lambda_1=1e-3
        )
        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(
c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
     district         rmse  log_marginal_likelihood  \
0  AHMEDNAGAR  1147.891774              -297.098709   

                                   coefficient_means  \
0  [0.00017421912766629884, 0.0001022266202467690...   

                                    coefficient_stds  intercept_mean  \
0  [0.010292224337625017, 0.010291754452258777, 0...     5655.984167   

   intercept_std  effective_params  differencing  n_iter  
0            NaN      1.299915e+10             0     500  

=== Forecast Data ===
              district       date  actual  forecast_mean  forecast_std
date                                                                  
2020-07-01  AHMEDNAGAR 2020-07-01  6225.0    5658.771976   1181.417269
2020-08-01  AHMEDNAGAR 2020-08-01  6609.0    5658.745849   1182.084431
2020-09-01  AHMEDNAGAR 2020-09-01  7268.0    5658.646842   1182.533193
2020-10-01  AHMEDNAGAR 2020-10-01  6799.0    5659.056467   1183.212084
2020-11-01  AHMEDNAGAR 2020-11-01  5248.0    565

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(
c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
   district        rmse  log_marginal_likelihood  \
0  AMRAVATI  527.651504              -239.703052   

                                   coefficient_means  \
0  [-0.007606445821520719, 0.0030314872396111966,...   

                                    coefficient_stds  intercept_mean  \
0  [0.04252063062931969, 0.04250811006620522, 0.0...       24.378582   

   intercept_std  effective_params  differencing  n_iter  
0            NaN      4.041172e+07             1     500  

=== Forecast Data ===
            district       date  actual  forecast_mean  forecast_std
date                                                                
2020-07-01  AMRAVATI 2020-07-01    2772    3202.748144    280.387128
2020-08-01  AMRAVATI 2020-08-01    3081    3223.599557    280.882549
2020-09-01  AMRAVATI 2020-09-01    3483    3240.977250    280.463103
2020-10-01  AMRAVATI 2020-10-01    3505    3252.171338    280.963113
2020-11-01  AMRAVATI 2020-11-01    3484    3284.033425    279.4565

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(
c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


...

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(
c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
  district        rmse  log_marginal_likelihood  \
0   WARDHA  204.713415              -229.593031   

                                   coefficient_means  \
0  [-0.05018692399133392, -0.020814263103176526, ...   

                                    coefficient_stds  intercept_mean  \
0  [0.06952992710072865, 0.06961560276363121, 0.0...       20.624198   

   intercept_std  effective_params  differencing  n_iter  
0            NaN      7.225715e+06             1     500  

=== Forecast Data ===
           district       date  actual  forecast_mean  forecast_std
date                                                               
2020-07-01   WARDHA 2020-07-01    1131    1106.092989    205.481229
2020-08-01   WARDHA 2020-08-01    1326    1120.936332    205.465626
2020-09-01   WARDHA 2020-09-01    1431    1130.945173    205.563393
2020-10-01   WARDHA 2020-10-01    1425    1139.296348    205.720990
2020-11-01   WARDHA 2020-11-01    1342    1158.770466    205.639150
=== Me

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(
c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
   district        rmse  log_marginal_likelihood  \
0  YAVATMAL  500.346237              -268.358064   

                                   coefficient_means  \
0  [0.13610079067647174, 0.0014311377579161837, -...   

                                    coefficient_stds  intercept_mean  \
0  [0.09438222069989387, 0.09364014843948175, 0.0...     2899.098963   

   intercept_std  effective_params  differencing  n_iter  
0            NaN      1.858053e+07             0     500  

=== Forecast Data ===
            district       date  actual  forecast_mean  forecast_std
date                                                                
2020-07-01  YAVATMAL 2020-07-01  4215.0    3124.008142    657.290905
2020-08-01  YAVATMAL 2020-08-01  3800.0    3315.079779    713.575405
2020-09-01  YAVATMAL 2020-09-01  3624.0    3267.196511    728.062166
2020-10-01  YAVATMAL 2020-10-01  3708.0    3212.340655    741.056735
2020-11-01  YAVATMAL 2020-11-01  3219.0    3185.697655    762.9369

{'district': 'MUMBAI SUBURBAN',
 'forecast_df':                    district       date  actual  forecast_mean  forecast_std
 date                                                                       
 2021-02-01  MUMBAI SUBURBAN 2021-02-01    3958    5377.220479    881.390890
 2021-03-01  MUMBAI SUBURBAN 2021-03-01    4827    5485.373364    880.705417,
 'metrics_df':           district         rmse  log_marginal_likelihood  \
 0  MUMBAI SUBURBAN  1106.264492               -40.993208   
 
                                    coefficient_means  \
 0  [0.0004577890775305479, -0.001090077856734878,...   
 
                                     coefficient_stds  intercept_mean  \
 0  [0.030557320589525618, 0.030554889196201532, 0...      106.331803   
 
    intercept_std  effective_params  differencing  n_iter  
 0            NaN      8.247671e+08             1     500  ,
 'model': BayesianRidge(alpha_1=1e-05, compute_score=True, lambda_1=0.001, n_iter=500),
 'coef_means': array([ 0.00045779